In [1]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [2]:
import cv2

In [3]:
import numpy as np

In [4]:
import tensorflow
from tensorflow import keras
print(tensorflow.__version__)

2.3.0


In [5]:
physical_devices = tensorflow.config.list_physical_devices('GPU')
for gpu in physical_devices:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

Name: /physical_device:GPU:0   Type: GPU


In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13277661126827590963
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 1606181354862876875
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2196032718
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16790316330811513960
physical_device_desc: "device: 0, name: GeForce GTX 1060 3GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5251266043099861140
physical_device_desc: "device: XLA_GPU device"
]


In [7]:
# Reference : https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html

In [8]:
if physical_devices != []:
    tensorflow.config.experimental.set_memory_growth(physical_devices[0], True)
    physical_devices[0]

In [9]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.applications.imagenet_utils import decode_predictions

In [10]:
inputShape = (299, 299)
preprocess = preprocess_input

In [11]:
model = Xception(weights="imagenet")

In [12]:
#video_file = "Jellyfish.mp4"
video_file = "https://www.sample-videos.com/video123/mp4/720/big_buck_bunny_720p_10mb.mp4"


In [13]:
cap = cv2.VideoCapture(video_file)

In [14]:
fourcc = cv2.VideoWriter_fourcc(*'avc1')
out = cv2.VideoWriter('output.mkv',fourcc, 30.0, (1920,1080))

In [15]:
# Ref: https://stackoverflow.com/questions/40273109/convert-python-opencv-mat-image-to-tensorflow-image-data/40273815

In [16]:
while(cap.isOpened()):
    ret, frame = cap.read() 
    
    if ret == True:
        image = cv2.resize(frame,dsize=(inputShape[0],inputShape[1]), interpolation = cv2.INTER_CUBIC) 
        np_image_data = np.asarray(image)

        image = np.expand_dims(image, axis=0)

        image = preprocess(image)

        preds = model.predict(image)
        P = decode_predictions(preds)    

        cv2.putText(frame, str(P[0][0][1]) + ":" + str(P[0][0][2]), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (126, 126, 126), 2)
        
        image_write = cv2.resize(frame,dsize=(1920,1080), interpolation = cv2.INTER_CUBIC)
        out.write(image_write)  

        cv2.imshow('frame',frame)        
    
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()